# Random Shaking Volume Denoising (RSVD) using opticalflow3d

Usando el phantom creado por Antonio, encontramos una configuración ad-hoc de los diferentes parámetros de filtrado, y comenamos a variar uno a uno los parámetros para identificar su impacto.

In [ ]:
local_debug = True

In [ ]:
vol_filename = "small_vol.mrc"

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    print("Running in Colab")
    !pip install cupy-cuda12x
    !pip install opticalflow3D
    !apt install libcudart11.0
    !apt install libcublas11
    !apt install libcufft10
    !apt install libcusparse11
    !apt install libnvrtc11.2
    #from google.colab import drive
    #drive.mount('/content/drive')
    #!cp drive/Shareddrives/TomogramDenoising/tomograms/{vol_name}.tif .
else:
    print("Running in locahost")
    #!cp ~/Downloads/{vol_name}.tif .

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import logging
import mrcfile
import skimage

In [ ]:
import opticalflow3D
import warnings
from numba.core.errors import NumbaPerformanceWarning

warnings.filterwarnings("ignore", category=NumbaPerformanceWarning)

In [ ]:
if local_debug:
    !ln -sf ../../motion_estimation/src/motion_estimation .
else:
    !pip install "denoising @ git+https://github.com/vicente-gonzalez-ruiz/motion_estimation"

In [ ]:
if local_debug:
    !ln -sf denoising/volume/RSVD.py .
else:
    !pip install "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
import RSVD

In [ ]:
if local_debug:
    !ln -sf ../../information_theory/src/information_theory/ .
else:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
import information_theory  # pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
vol_filename = "phantom01_noise_2_0.tif"

In [ ]:
from collections import namedtuple
Args = namedtuple("args", ["input", "output"])
fn, fe = vol_filename.split(".")
args = Args(vol_filename , fn + "_denoised." + fe)
print(args)

In [ ]:
%%bash -s "$args.input"
set -x
OUTPUT_FILENAME=$1
#rm -f $OUTPUT_FILENAME
if test ! -f $OUTPUT_FILENAME ; then
    FILEID="1MZ05sO6lLDEvZRwSEzhSmm3kbaIuhvlq" # https://drive.google.com/file/d/1MZ05sO6lLDEvZRwSEzhSmm3kbaIuhvlq/view?usp=drive_link
    #FILEID="1iui018SGKa5nb0ybeoUAB7uCUhl9EJ5o" #https://drive.google.com/file/d/1iui018SGKa5nb0ybeoUAB7uCUhl9EJ5o/view?usp=sharing
    #wget --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O $OUTPUT_FILENAME #2> /dev/null
    ~/envs/OF3D/bin/gdown https://drive.google.com/uc?id=$FILEID # pip install gdown
fi
ls -l $OUTPUT_FILENAME
set +x

In [ ]:
noisy = skimage.io.imread(args.input, plugin="tifffile").astype(np.float32)
#stack_MRC = mrcfile.open(args.input)
#noisy = stack_MRC.data

In [ ]:
noisy = (255*(noisy - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.uint8)

In [ ]:
noisy.shape

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(16, 16))
slice_idx = noisy.shape[0]//2
axs.imshow(noisy[:, ::-1, :][slice_idx], cmap="gray")
axs.set_title(f"Noisy")
fig.tight_layout()
plt.show()

In [ ]:
# Configuración por defecto
N_iters = 5 # Main parameter of the denoiser. More iterations generates a higher denoising, but also (potentially) a higher blurring. Time grows O(N_iters). Default: 25
RS_std_dev = 2.0 # Standard deviation of the displacements produced by the random shaking. Controls (linearly) the blurring. Cannot be 0.
                 # Negligible impact in the running time. RS_std_dev and pyramid_levels are dependent of each other. Default: 2.0
pyramid_levels = 3 # Number of levels of the gaussian pyramid used by the Farneback algorithm to manage large displacements. It should be large enough to arhieve that
                   # the average of the absolute value of shaking displacements is not smaller than the average of the absolute value of the OF displacements. Default: 3.
window_side = 5
N_poly = 9
iterations = 5 # Number of iterations of the Farneback algorithm. The higher, the better (always for convergence of the algorith), but also requires more time (linearly). At least 1 iteration is required. Default: 5.
block_size = (noisy.shape[0], noisy.shape[1], noisy.shape[2]) # Depends on the memory of the GPU
denoiser = RSVD.Random_Shaking_Denoising(logging_level=logging.DEBUG)
denoised = denoiser.filter_volume(
    noisy,
    std_dev=RS_std_dev,
    window_side=window_side,
    N_poly=N_poly,
    N_iters=N_iters,
    iterations=iterations,
    pyramid_levels=pyramid_levels,
    block_size=block_size)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
slice_idx = noisy.shape[0]//2
imgplot = plt.imshow(noisy[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_std_dev}")
plt.imshow(denoised[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[slice_idx][::-1, :] - denoised[slice_idx][::-1, :], cmap="gray")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
slice_idx = noisy.shape[1]//2
axs[0].imshow(noisy[:, slice_idx], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, slice_idx], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, slice_idx], denoised[:, slice_idx])})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
slice_idx = noisy.shape[2]//2
axs[0].imshow(noisy[:, :, slice_idx], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, :, slice_idx], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, :, slice_idx], denoised[:, :, slice_idx])})")
fig.tight_layout()
plt.show()

In [ ]:
input()

In [ ]:
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=33, presmoothing=None, filter_type="gaussian", filter_size=3)
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=13, presmoothing=None, filter_type="gaussian", filter_size=7)
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=5, presmoothing=3, filter_type="gaussian", filter_size=5)
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=3, presmoothing=5, filter_type="box", filter_size=3)
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=5, scale=0.5, spatial_size=3, presmoothing=5, filter_type="box", filter_size=3)
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=5, scale=0.5, spatial_size=3, presmoothing=5, filter_type="gaussian", filter_size=11)
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=1, scale=0.5, spatial_size=5, presmoothing=4, filter_type="box", filter_size=5)
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=5, scale=0.5, spatial_size=9, sigma_k=0.15, filter_type="box" ,filter_size=21, presmoothing=None, device_id=0) # Default values
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=5, scale=0.5, spatial_size=9, sigma_k=1.15, filter_type="gaussian" ,filter_size=11, presmoothing=None, device_id=0)
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=5, scale=0.5, spatial_size=9, sigma_k=0.5, filter_type="gaussian" ,filter_size=11, presmoothing=None, device_id=0)
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=5, scale=0.5, spatial_size=5, sigma_k=0.5, filter_type="gaussian" ,filter_size=11, presmoothing=None, device_id=0); RS_sigma = 1.5
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=5, scale=0.5, spatial_size=5, sigma_k=0.75, filter_type="gaussian" ,filter_size=11, presmoothing=None, device_id=0); RS_sigma = 1.5
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=5, scale=0.5, spatial_size=5, sigma_k=1.0, filter_type="gaussian" ,filter_size=9, presmoothing=None, device_id=0); RS_sigma = 1.5
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=5, scale=0.5, spatial_size=6, sigma_k=1.0, filter_type="gaussian" ,filter_size=9, presmoothing=None, device_id=0); RS_sigma = 1.5

def show_image(denoised_volume, title):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title)
    #axs.imshow(denoised_volume[denoised_volume.shape[0]//2].astype(np.uint8), cmap="gray")
    axs.imshow(denoised_volume[denoised_volume.shape[0]//2], cmap="gray")
    plt.show()

def get_quality(noisy, denoised):
    slice_idx = noisy.shape[0]//2
    return information_theory.information.compute_quality_index(noisy[slice_idx], denoised[slice_idx])

denoiser = RSVD.Random_Shaking_Denoising(logging_level=logging.INFO, show_image=show_image, get_quality=get_quality)


In [ ]:
RS_sigma = 2.0
N_iters = 25
window_side = 5
N_poly = 9
denoised = denoiser.filter_volume(noisy, std_dev=RS_sigma, window_side=window_side, N_poly=N_poly, N_iters=N_iters, block_size=block_size)

#denoised = RSIVD.filter(farneback, block_size, noisy, RS_sigma=RS_sigma, N_iters=20)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
slice_idx = noisy.shape[0]//2
axs[0].imshow(noisy[slice_idx], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[slice_idx], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[slice_idx], denoised[slice_idx])})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
slice_idx = noisy.shape[1]//2
axs[0].imshow(noisy[:, slice_idx], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, slice_idx], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, slice_idx], denoised[:, slice_idx])})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
slice_idx = noisy.shape[2]//2
axs[0].imshow(noisy[:, :, slice_idx], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, :, slice_idx], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, :, slice_idx], denoised[:, :, slice_idx])})")
fig.tight_layout()
plt.show()

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
slice_idx = noisy.shape[0]//2
imgplot = plt.imshow(noisy[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[slice_idx][::-1, :] - denoised[slice_idx][::-1, :], cmap="gray")

In [ ]:
with mrcfile.new(f"{fn}_{RS_sigma}_{N_iters}.mrc", overwrite=True) as mrc:
            mrc.set_data(denoised.astype(np.float32))
            mrc.data
#skimage.io.imsave(f"{args.output}_{RS_sigma}_{N_iters}.tif", denoised, imagej=True)
f"{fn}_{RS_sigma}_{N_iters}.mrc"

In [ ]:
input()

In [ ]:
denoiser = RSVD.Random_Shaking_Denoising(logging_level=logging.WARNING, show_image=show_image, get_quality=get_quality)
RS_sigma = 1.0
N_iters = 25
window_side = 5
N_poly = 9
denoised = denoiser.filter_volume(noisy, std_dev=RS_sigma, window_side=window_side, N_poly=N_poly, N_iters=N_iters, block_size=block_size)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
slice_idx = noisy.shape[0]//2
imgplot = plt.imshow(noisy[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[slice_idx][::-1, :] - denoised[slice_idx][::-1, :], cmap="gray")

In [ ]:
denoiser = RSVD.Random_Shaking_Denoising(logging_level=logging.WARNING, show_image=show_image, get_quality=get_quality)
RS_sigma = 3.0
N_iters = 25
window_side = 5
N_poly = 9
denoised = denoiser.filter_volume(noisy, std_dev=RS_sigma, window_side=window_side, N_poly=N_poly, N_iters=N_iters, block_size=block_size)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
slice_idx = noisy.shape[0]//2
imgplot = plt.imshow(noisy[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[slice_idx][::-1, :] - denoised[slice_idx][::-1, :], cmap="gray")

In [ ]:
denoiser = RSVD.Random_Shaking_Denoising(logging_level=logging.WARNING, show_image=show_image, get_quality=get_quality)
RS_sigma = 5.0
N_iters = 25
window_side = 5
N_poly = 9
denoised = denoiser.filter_volume(noisy, std_dev=RS_sigma, window_side=window_side, N_poly=N_poly, N_iters=N_iters, block_size=block_size)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
slice_idx = noisy.shape[0]//2
imgplot = plt.imshow(noisy[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[slice_idx][::-1, :] - denoised[slice_idx][::-1, :], cmap="gray")

In [ ]:
denoiser = RSVD.Random_Shaking_Denoising(logging_level=logging.WARNING, show_image=show_image, get_quality=get_quality)
RS_sigma = 5.0
N_iters = 25
window_side = 5
N_poly = 9
pyramid_levels = 4
denoised = denoiser.filter_volume(noisy, std_dev=RS_sigma, window_side=window_side, pyramid_levels=pyramid_levels, N_poly=N_poly, N_iters=N_iters, block_size=block_size)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
slice_idx = noisy.shape[0]//2
imgplot = plt.imshow(noisy[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[slice_idx][::-1, :] - denoised[slice_idx][::-1, :], cmap="gray")

In [ ]:
denoiser = RSVD.Random_Shaking_Denoising(logging_level=logging.WARNING, show_image=show_image, get_quality=get_quality)
RS_sigma = 5.0
N_iters = 25
window_side = 5
N_poly = 9
pyramid_levels = 1
denoised = denoiser.filter_volume(noisy, std_dev=RS_sigma, window_side=window_side, pyramid_levels=pyramid_levels, N_poly=N_poly, N_iters=N_iters, block_size=block_size)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
slice_idx = noisy.shape[0]//2
imgplot = plt.imshow(noisy[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[slice_idx][::-1, :] - denoised[slice_idx][::-1, :], cmap="gray")

In [ ]:
denoiser = RSVD.Random_Shaking_Denoising(logging_level=logging.WARNING, show_image=show_image, get_quality=get_quality)
RS_sigma = 2.5
N_iters = 25
window_side = 5
N_poly = 9
pyramid_levels = 1
denoised = denoiser.filter_volume(noisy, std_dev=RS_sigma, window_side=window_side, pyramid_levels=pyramid_levels, N_poly=N_poly, N_iters=N_iters, block_size=block_size)
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
slice_idx = noisy.shape[0]//2
imgplot = plt.imshow(noisy[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[slice_idx][::-1, :] - denoised[slice_idx][::-1, :], cmap="gray")

In [ ]:
input()

In [ ]:
# Impact of RS_sigma
for i in range(10):
    RS_sigma = (i + 1)/

In [ ]:
skimage.io.imsave(f"{vol_name}_denoised_{RS_sigma}.tif", denoised_vol, imagej=True)

In [ ]:
img = skimage.io.imread("http://www.hpca.ual.es/~vruiz/images/barb.png")#[:256, :256]

In [ ]:
noisy_vol = np.stack([img]*32)

In [ ]:
noisy_vol.shape

In [ ]:
mean = 0
var = 1000
sigma = var**0.5
for i in range(noisy_vol.shape[0]):
    noise = np.random.normal(mean,sigma,img.shape).reshape(img.shape)
    noisy_vol[i] = np.clip(a=img.astype(np.float32) + noise, a_min=0, a_max=255).astype(np.uint8)

In [ ]:
farneback = opticalflow3D.Farneback3D(iters=5,
                                      num_levels=3,
                                      scale=0.5,
                                      spatial_size=5,
                                      presmoothing=2,
                                      filter_type="box",
                                      filter_size=5,
                                     )

In [ ]:
RS_sigma = 1.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=30)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15] - denoised_vol[15], cmap="gray")

In [ ]:
RS_sigma = 1.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=100)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15] - denoised_vol[15], cmap="gray")

In [ ]:
RS_sigma = 2.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=30)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15] - denoised_vol[15], cmap="gray")

In [ ]:
RS_sigma = 3.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=30)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15] - denoised_vol[15], cmap="gray")

In [ ]:
#skimage.io.imsave(f"{vol_name}_denoised_{RS_sigma}.tif", denoised_vol, imagej=True)